In [1]:
import torch, torchvision
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

import matplotlib.colors as mc
import math
import matplotlib.pyplot as plt
import glob
from PIL import Image, ImageDraw
import argparse
import sys

In [2]:


from math import sqrt

COLORS = (
    (255,255,255),
    (255,0,0),
    (0,255,0),
    (0,0,255),
    (255,255,0),
    (0,255,255),
    (255,0,255),
    (192,192,192),
    (128,128,128),
    (128,0,0),
    (128,128,0),
    (0,128,0),
    (128,0,128),
    (0,128,128),
    (0,0,128)
)

def closest_color(rgb):
    r, g, b = rgb
    color_diffs = []
    for color in COLORS:
        cr, cg, cb = color
        color_diff = sqrt(abs(r - cr)**2 + abs(g - cg)**2 + abs(b - cb)**2)
        color_diffs.append((color_diff, color))
    return min(color_diffs)[1]

def get_colors(image_file, numcolors=10, resize=150):
    # Resize image to speed up processing
    img = Image.open(image_file)
    img = img.copy()
    img.thumbnail((resize, resize))

    # Reduce to palette
    paletted = img.convert('P', palette=Image.ADAPTIVE, colors=numcolors)

    # Find dominant colors
    palette = paletted.getpalette()
    color_counts = sorted(paletted.getcolors(), reverse=True)
    colors = list()
    for i in range(numcolors):
        palette_index = color_counts[i][1]
        dominant_color = palette[palette_index*3:palette_index*3+3]
        colors.append(tuple(dominant_color))

    return colors

def get_mask_image(mask, box,image):
    mask_h = int(math.ceil(box[3] - box[1]))
    mask_w = int(math.ceil(box[2] - box[0]))

    original_img = np.array(Image.fromarray(image))

    temp_mask = np.zeros((mask_h, mask_w))

    for h_idx in range(int(box[1]), int(box[3])):
        for w_idx in range(int(box[0]), int(box[2])):
            temp_mask[h_idx - int(box[1])][w_idx - int(box[0])] = mask[h_idx][w_idx]

    temp_masks_ints = temp_mask.astype(int)

    temp_mask_fill = np.zeros((mask_h, mask_w, 4))
    for h_idx, h_bw in enumerate(temp_masks_ints):
        for w_idx, w_bw in enumerate(h_bw):
            if (w_bw == 0):
                temp_mask_fill[h_idx][w_idx] = [83,62,65,0.2]
            else:
                orig_w = int(math.ceil(w_idx + box[0]))
                orig_h = int(math.ceil(h_idx + box[1]))
                temp_mask_fill[h_idx][w_idx] = np.append(original_img[orig_h][orig_w],1)

    return temp_mask_fill

In [3]:
def find_dominant_color(filename):
    #Resizing parameters
    width, height = 150,150
    image = Image.open(filename)
    image = image.resize((width, height),resample = 0)
    #Get colors from image object
    pixels = image.getcolors(width * height)
    #Sort them by count number(first element of tuple)
    sorted_pixels = sorted(pixels, key=lambda t: t[0])
    #Get the most frequent color
    dominant_color = sorted_pixels[-2][1]
    return dominant_color

In [10]:
def store_color_sketch_from_masks(image, video_id, keyframe_id):
    im1 = Image.open(image) 
    im_quantized = im1.quantize(25)
    im_quantized.save("quantized.png")
    detectron2_img = cv2.imread("quantized.png")
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
    # Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
    predictor = DefaultPredictor(cfg)
    outputs = predictor(detectron2_img)

    masks = outputs["instances"].pred_masks.cpu().numpy()
    boxes = outputs["instances"].pred_boxes.tensor.cpu().numpy()
    
    masks_arr = []
    
    for index, mask in enumerate(masks):
        temp_mask_fill = get_mask_image(mask, boxes[index], detectron2_img)
        cv2.imwrite("temporary_image.png",temp_mask_fill[:,:,0:3])
        Image.open("temporary_image.png")
        dominant_color = list(find_dominant_color("temporary_image.png"))
        box = boxes[index].tolist()
        ###### cottontail db logic ######
        # store rgb, border boxes, keyframe id and video id in database 
        #os.remove("temporary_image.png")

In [5]:
def get_all_filesname(path):
    files = os.listdir(path)
    return files

def get_keyframe_id(filename, video_id,path):
    video_path = filename.replace(f"{path}/home/keyframes_filtered/","")
    keyframe_path = video_path.replace(video_id,"")
    keyframe_id = keyframe_path.replace(f"shot_","").replace(f"_RKF.png","")
    return keyframe_id

# change this path according to your computer
path = "/run/user/1000/gvfs/dav:host=tal.diskstation.me,port=5006,ssl=true"


In [6]:
def run(path):
    video_filelist = sorted(get_all_filesname(f"{path}/home/keyframes_filtered"))
    for videonr in video_filelist:
        for filename in get_all_filesname(f"{path}/home/keyframes_filtered/{videonr}"):
            keyframe_id = videonr,get_keyframe_id(filename,videonr,path)
            image = f"{path}/home/keyframes_filtered/{videonr}/{filename}"
            try:
                store_color_sketch_from_masks(image, videonr, keyframe_id)
            except:
                print(filename)
            print("keyframe done")

In [11]:
run(path)

keyframe done
shot00032_11_RKF.png
keyframe done
keyframe done
keyframe done
keyframe done
keyframe done
keyframe done
keyframe done
keyframe done
keyframe done
keyframe done
keyframe done
keyframe done
keyframe done
keyframe done
keyframe done
keyframe done
keyframe done
keyframe done
keyframe done
keyframe done


In [15]:
d = {}
for i in range(14):
    d[i] = []
    for j in range(10):
        if j%2 == 0:
            d[i].append(j)

In [18]:
with open("test.json", "w") as fi:
    fi.write(json.dumps(d))